# Решение уравнения теплопроводности

В 0$^{ом}$ приближении рассматриваем воду без вязкости. В таком случае имеем уравнение:
$$ \rho c(T) \frac{\partial T}{\partial t} = \frac{\partial}{\partial x} \left(\kappa(T) \frac{\partial T}{\partial x}\right) + Q(x, t) $$
где $\rho$ - плотность, $c(T)$- теплоемкость, $\kappa(T)$- коэффициент теплопроводности, $Q$- источник нагрева.


$c(Т), \kappa(T)$  будем брать из интерполяции табличных данных, пока положим константами

## Построение разностной схемы

Достаточно тривиально, но на всякий случай запишу это здесь

$x = i*h, i = 0, 1, 2, ..., N$; $t= j*\tau, j = 0, 1, 2, ..., M$

$$\rho c \frac{T^{j+1}_i - T^{j}_i}{\tau} =\kappa \frac{T^{j}_{i+1} - 2T^{j}_i + T^{j}_{i-1}}{h^2} + Q^j_i$$

Граничные условия могут быть заданы следующим образом:
$T(0, x) = T_1(x)$ , $T(t, 0) = T_2(t)$ , $T`_x(t, l) = T_3(t)$

$$T^{0}_{i} = T_1(ih)$$
$$T^{j}_{0} = T_2(j\tau)$$
$$\frac{T^{j}_i - T^{j}_{i-1}}{h} = T_3(j\tau)$$

In [19]:
import numpy as np

#define constants
T0 = 300 #K
C = 4200 #J/(K kg)
RHO = 1000 # kg/m^3
KAPPA = 1e8 # m^2/s
X_MAX = 1 # m
T_MAX = 1 # s
T_IMP = 0.1 # s
P = 1 # J/s
W = 1e-2 # m
Q_LEAVE = 1 # J/(c m^2)

N = 20
M = 20
h = X_MAX/N
tau = T_MAX/M


In [20]:
def T1(x):
    return T0
def T2(t):
    if(t > T_IMP):
        t = T_IMP
    return T0 + 100*t/T_IMP
def T3(t):
    return Q_LEAVE

def Q(x, t):
    if(t > T_IMP):
        return 0
    return 2*P/(np.pi*W**2)*np.exp(-2*abs(x)**2/(W**2))

In [21]:
T = [[0 for i in range(0, N)] for j in range(0, M)]
for i in range(0, N):
    T[0][i] = T1(i*h)
    
for j in range(0, M):
    T[j][0] = T2(j*tau)
    

In [23]:
C__ = tau/RHO/C
C_ = C__*KAPPA/h**2
 
def calc(j, i):
    T[j+1][i] = T[j][i] + C_*(T[j][i+1] - 2*T[j][i] + T[j][i-1]) + C__*Q(i*h, j*tau)

for j in range(0, M-1):
    for i in range(1, N-1):
        calc(j, i)
    T[j+1][N-1] = T[j+1][N-2] + h*T3(j*tau)
        
T

[[300.0,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300,
  300],
 [350.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.05],
 [400.0,
  24109.523809523806,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  323.8095238095292,
  323.8595238095292],
 [400.0,
  -22604008.39002267,
  11338168.48072562,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  11637.868480728186,
  -10990.24943310913,
  -10990.19943310913],
 [400.0,
  26904341353.88186,
  -21550588189.36399,
  5398985290.821723,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  300.0,
  5399284.990822945,
  -1616264